In [64]:
import datetime
import numpy as np
import pandas as pd
import os
import sys
from importlib import reload
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
sys.path.append("..")

In [2]:
import lc_investing.rwm
lc_investing.rwm = reload(lc_investing.rwm)

import lc_investing.constant_percent_stock
lc_investing.constant_percent_stock = reload(lc_investing.constant_percent_stock)

In [145]:
t0=time()

s2 = lc_investing.constant_percent_stock.Simulation(#data_folder='/content/drive/Othercomputers/My MacBook Air/Taxes_and_other_forms/lifecycle_investing/lc_investing/data/',
                                                    data_folder='..\\lc_investing\\data\\',
                                                    bondadj=0,
                                                    #margadj=marg_adj,
                                                    stockadj=0,
                                                    #lambdacons=labmdacons, # stock allocation
                                                    start_age=30,
                                                    start_amt=1,
                                                    #max_rate = r,
                                                    #borrowing_rate_override=borr_rate
)
print(round((time()-t0)/60, 3), 'mins')


0.113 mins


In [139]:
M = 100

R_A = 1+s2.monthly_data.loc[0:1661, ['Months_beginning_Jan_1871',
                        'Monthly_real_gov_bond_rate',
                        'Monthly_real_margin_rate', 
                        'Monthly_real_stock_rate']].values[:529,1:].reshape(-1,3)

# multiple cohorts
R_A = np.repeat(R_A[np.newaxis,:,:], M, axis=0)

p = np.zeros((R_A.shape[0], R_A.shape[1], 1))
fv_0 = 1   # start with $1
W = np.repeat([[[0,0,1]]], R_A.shape[1], axis=1)
W = np.repeat(W, M, axis=0)

print(p.shape)
print(R_A.shape)
print(W.shape)

(100, 529, 1)
(100, 529, 3)
(100, 529, 3)


In [144]:
def compute_rwm(p: np.array, 
                fv_0: float, 
                R_A: np.array, 
                W: np.array, 
                funcP=None, 
                funcW=None):
    """
    Compute returns using matrices
    """
    
    assert p.ndim==R_A.ndim==W.ndim==3, 'Must be 3-d arrays'
    assert R_A.shape[2]==3, 'Must be three assets'
    assert p.shape[0]==R_A.shape[0]==W.shape[0], 'Number of cohorts must be equal'

    p[0] = fv_0
    T = R_A.shape[1]
    M = R_A.shape[0]
    
    for t in range(0,T):
        if t%50==0: print(t)
        if t>1:
            pass
            #p[:,t,0]=5
            #W[:,t,:] = np.array([0,0,1]) #funcW()
        P = np.triu(np.matmul(p[:,0:t+1,:], np.ones((M,1,t+1))))   # use t+1 to reduce computation
        #print(P.shape)
        
        R_P = np.matmul(R_A[:,0:t+1,:], np.transpose(W, axes=(0,2,1))[:,:,0:t+1])  # keep t+1 columns to reduce computation
        #print('R_P', R_P.shape)
        
        
        r_t = np.diagonal(R_P, axis1=1, axis2=2).reshape(-1, t+1, 1)  # keep t+1 columns to reduce computation
        #print('r_t',r_t.shape)
        
        R_t = np.matmul(np.ones((M,t+1,1)), np.transpose(r_t, (0,2,1)))  # keep t+1 columns to reduce computation
        #print('R_t', R_t.shape)
        
        # set lower triangle to 1
        I,J = np.tril_indices(t+1, -1) # keep t+1 columns to reduce computation
        R_t[:,I,J]=1
        
        R = np.cumprod(R_t, axis=2)
        #print('R',R.shape)
#         if t==1: 
#             raise Exception('RH')
              
        #print('P^T', np.transpose(P[:,t:t+1,:], (0,2,1)).shape)
        FV = np.diagonal(np.matmul(np.transpose(P[:,:,t:t+1], (0,2,1)), R[:,:,t:t+1]))
        #print('FV', FV.shape)
        
    
    return FV
t0=time()
out = compute_rwm(p, fv_0, R_A, W)
print(round((time()-t0)/60, 3), 'mins')
print(out.shape)

# 1 min = 20 cohorts

0
50
100
150
200
250
300
350
400
450
500
2.862 mins
(1, 1)


In [143]:
out

array([[2586.96341039]])